In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from collections import Counter

import torch

/local/diwu/miniconda3/envs/repoeval/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_dir = '/home/diwu/ralm/self-rag/data_creation/'

gen_data = [json.loads(line) for line in open(data_dir + '/generator_training_data_full_output_1005.jsonl').readlines()]
critic_data = json.load(open(data_dir + '/critic_training_data_gpt4_reward_all_0813_train.json'))

critic_tasks = set([x['task'] for x in critic_data])
critic_task2data = {t: [x for x in critic_data if x['task'] == t] for t in critic_tasks} 
# gen_task2data = {t: [x for x in gen_data if x['task'] == t] for t in critic_tasks} 
# gen_data[0]

In [3]:
critic_retrieval_pos = [x for x in critic_task2data['retrieval'] if x['output'] == '[Retrieval]']
critic_retrieval_neg = [x for x in critic_task2data['retrieval'] if x['output'] == '[No Retrieval]']
gen_retrieval_pos = [x for x in gen_data if '[Retrieval]' in x['output']]
gen_retrieval_neg = [x for x in gen_data if '[Retrieval]' not in x['output']]
len(critic_retrieval_pos), len(critic_retrieval_neg), len(gen_retrieval_pos), len(gen_retrieval_neg)

(2366, 2432, 74219, 71400)

In [33]:
start = 400
end = 500

[x['input'] for x in critic_retrieval_neg[start:end]]

['Task instruction: Formulate a conclusion based on the following statement The majority of students passed the exam\n',
 "Task instruction: Create a poem that explains the wildflower's resilience.\n",
 'Task instruction: Generate a question from the following sentence The Eiffel Tower was constructed in 1889.\n',
 'Task instruction: Create a chart displaying the following data. Duration of sleep in hours:\nPerson A: 6\nPerson B: 7\nPerson C: 8\n',
 'Task instruction: Generate an appropriate response if someone asks: "Do you have any advice for me?"\n',
 'Task instruction: Classify the following paragraph as formal or informal. Hey there! I was just wondering if you were available to meet up for lunch tomorrow?\n',
 'Task instruction: Open up a discussion about a given topic.\n',
 'Task instruction: Identify the incorrect word in this sentence. The singer joyfully belted their solo.\n',
 'Task instruction: Provide a summary for the passage given below. Artificial intelligence (AI) is a

In [4]:
import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="7"


# for each no-retrieval instance, retrieve the 5 most similar retrieval instances and compare their difference
# are there any features that could be added to flip the label?


from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer, util
import numpy as np
from typing import List, Tuple

def jaccard_similarity(str1: str, str2: str) -> float:
    """Calculate the Jaccard Similarity between two strings."""
    set1 = set(str1.split())
    set2 = set(str2.split())
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    return len(intersection) / len(union)

def cosine_similarity_with_embedding(model, documents: List[str], query: str) -> List[Tuple[str, float]]:
    """Calculate cosine similarity using sentence transformer embeddings."""
    
    doc_embeddings = model.encode(documents)
    query_embedding = model.encode(query)
    print(doc_embeddings.shape, query_embedding.shape)
    similarities = util.cos_sim(query_embedding, doc_embeddings).squeeze()
    # similarities = cosine_similarity(query_embedding, doc_embeddings).flatten()
    return [(documents[i], similarities[i]) for i in range(len(documents))]

# def top_k_similar_strings(strings: List[str], query: str, k: int, similarity_func) -> List[Tuple[str, float]]:
#     """Return top-k most similar strings based on a similarity function."""
#     similarities = [(string, similarity_func(query, string)) for string in strings]
#     sorted_similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
#     return sorted_similarities[:k]

def top_k_similar_strings(model, strings: List[str], query: str, k: int, similarity_func) -> List[Tuple[str, float]]:
    """Return top-k most similar strings based on a similarity function."""
    similarities = similarity_func(model, strings, query)
    sorted_similarities = sorted(similarities, key=lambda x: x[1], reverse=True)
    return sorted_similarities[:k]


# Example usage
# inst2entry = {x['input']: x for x in critic_task2data['retrieval']}
# strings = [x['input'] for x in critic_task2data['retrieval']]
# query = '### Instruction:\nQuestion: Tell me a bio of {entity}.\n\n### Response:\n'
k = 20

# Jaccard similarity
# top_k_jaccard = top_k_similar_strings(strings, query, k, jaccard_similarity)
# print(top_k_jaccard)

# # Cosine similarity (this might take a bit longer due to model loading and embedding computation)
# top_k_cosine = top_k_similar_strings(strings, query, 5, cosine_similarity_with_embedding)
# print(strings)

model = SentenceTransformer('all-mpnet-base-v2')

def strip_input(x):
    sents = x.split('. ')
    if len(sents) <= 1:
        return x
    else:
        return '. '.join(sents[:-1])

for query in [strip_input(x['input'].replace('Task instruction:', '').strip()) for x in critic_retrieval_neg[50:100]]:
    strings = [strip_input(x['input'].replace('Task instruction:', '').strip()) for x in critic_retrieval_pos]
    top_k_cosine = top_k_similar_strings(model, strings, query, 5, cosine_similarity_with_embedding)
    print('Query', '\n\t', query)
    print('Most similar positive instances')
    print('\t', top_k_cosine)
    # break



(2366, 768) (768,)
Query 
	 Collect and organize the important points from the following text. The Hubble Space Telescope is a space telescope which is used to observe the universe from outside of Earth's atmosphere. It was launched in 1990 and is still in use today
Most similar positive instances
	 [('Explain the purpose of the following device: an astrolabe.', tensor(0.4823)), ('Explain the purpose of a satellite in two sentences.', tensor(0.4293)), ('What was the name of the first satellite to be sent into space?', tensor(0.4210)), ('What are the important contributions of NASA to space exploration?', tensor(0.3886)), ('Find out what is the closest star to Earth.', tensor(0.3531))]
(2366, 768) (768,)
Query 
	 Generate a funny joke.
Most similar positive instances
	 [('Generate a funny one-liner joke about cats.', tensor(0.6624)), ('Create an instance of a joke using a pun', tensor(0.6325)), ('Suggest a word that has the same meaning as “funny”.', tensor(0.5343)), ('Generate a movie 

In [32]:
import random

# Why not try applying goal-oriented clustering on the instructions?


# dump data
goal = "I would like to cluster these tasks based on their nature or type."
examples = ["math problems", "coding challenges", "commonsense reasoning", "creative writing", "grammar check"]
data_for_cluster = [strip_input(x['input'].replace('Task instruction:', '').strip()) for x in critic_retrieval_pos + critic_retrieval_neg]

sample_size = 200

out_data = {
    'goal': goal,
    "example_descriptions": examples,
    "texts": random.sample(data_for_cluster, sample_size)
}
# json.dump(out_data, open('selfrag_critic_tasks_for_clustering_{}samples.json'.format(sample_size), 'w'), indent=2)
# print('selfrag_critic_tasks_for_clustering_{}samples.json'.format(sample_size))

selfrag_critic_tasks_for_clustering_200samples.json


In [25]:
len(data_for_cluster)

4798

In [16]:
all_tasks_critic = [x['input'].replace('Task instruction:', '').strip() for x in critic_task2data['retrieval']]
all_tasks_critic[:5]

['Find the longest word in the sentence “Great minds think alike.”',
 "Explain why Einstein's theory of relativity is significant.",
 'Discover which word in the sentence carries the most weight. She ordered Chinese food for dinner.',
 "Make a smoothie recipe that's delicious and healthy.",
 'Create a hashtag related to this topic. Health and wellness']